# Segmenting and Clustering Neighborhoods in Toronto 

### Disclaimer

This notebook is structured in three parts - according to the three submissions on Coursera.

#### Install Libraries

In [1]:
#!conda install -c anaconda beautifulsoup4
#!conda install -c anaconda requests
#!conda install -c anaconda lxml
#!conda install -c conda-forge geocoder
#!conda install -c conda-forge folium

#### Import Libraries

In [2]:
from bs4 import BeautifulSoup
import requests
import lxml
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import geocoder
import folium
import json
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

## Part 1: Scrape Wikipedia page using BeautifulSoup

In this part we will scrape the Wikipedia table and duild a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name.

#### Get Table from Wikipedia

In [3]:
response = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050.')
soup = BeautifulSoup(response.text, 'lxml')
wiki_table = soup.find('table', 'wikitable sortable')

#### Extract Data from Wikipedia Table

In [4]:
pc_toronto = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighbourhood'])

PostalCode = []
Borough = []
Neighbourhood = []

for row in wiki_table.findChildren('tr'):
    cells = row.findChildren(['td', 'th'])
    
    if not cells[1].string == 'Not assigned' and not cells[1].string == 'Borough':
        # Postcode
        code = cells[0].string.strip()
        PostalCode.append(code)

        # Borough
        borough = cells[1].string.strip()
        Borough.append(borough)

        # Neighbourhood
        if not cells[2].string: 
            hood = cells[2].a.string.strip()
            Neighbourhood.append(hood)
        else: 
            hood = cells[2].string.strip()
            Neighbourhood.append(hood)
    
pc_toronto['PostalCode'] = PostalCode
pc_toronto['Borough'] = Borough
pc_toronto['Neighbourhood'] = Neighbourhood

print('The dataframe has', pc_toronto.shape[0], 'rows and', pc_toronto.shape[1], 'columns.')
print('\n')
print('The following Boroughs are in the dataframe:', pc_toronto['Borough'].unique())

The dataframe has 210 rows and 3 columns.


The following Boroughs are in the dataframe: ['North York' 'Downtown Toronto' 'Etobicoke' 'Scarborough' 'East York'
 'York' 'East Toronto' 'West Toronto' 'Central Toronto' 'Mississauga']


## Wrangle and clean data

#### Combine Postal Codes with Multiple Neighborhoods into one row

In [5]:
pc_toronto = pc_toronto.groupby(['PostalCode', 'Borough']).agg(lambda x: ', '.join(x)).reset_index()
pc_toronto.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


Verify the size of the dataframe

In [6]:
pc_toronto.shape

(103, 3)

## Part 2: Submit Notebook 2

In this part we will get the latitude and the longitude coordinates of each neighborhood and add them to our dataframe.

In [7]:
latitude = []
longitude = []

for x in pc_toronto['PostalCode']:
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.arcgis('{}, Toronto, Ontario'.format(x))
      lat_lng_coords = g.latlng

    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])

pc_toronto['Latitude'] = latitude
pc_toronto['Longitude'] = longitude

pc_toronto.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726276,-79.263625
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713054,-79.285055
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.724235,-79.227925
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696770,-79.259967


## Part 3: Explore and cluster the neighborhoods in Toronto

In this part we will explore the neighborhoods in Toronto and explore the kind of venues in these neighborhoods. We will then analyze and cluster the neighborhoods based on their most venue categories. Finally we're going to visualize the clusters.

### Explore Toronto's boroughs and neighbourhoods

#### Get the latitude and longitute of Toronto

In [8]:
g = geocoder.arcgis('Toronto, Ontario')
lat_lng_coords = g.latlng

lat_toronto = lat_lng_coords[0]
lng_toronto = lat_lng_coords[1]

#### Create a map of Toronto with neighbourhoods superimposed on top

In [9]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[lat_toronto, lng_toronto], zoom_start=10)

for lat, lng, bor, nei in zip(pc_toronto['Latitude'], pc_toronto['Longitude'], pc_toronto['Borough'], pc_toronto['Neighbourhood']):
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup='{} ({})'.format(nei, bor),
        color='#0090C1',
        fill=True,
        fill_color='#0090C1',
        fill_opacity=0.5).add_to(map_toronto)

map_toronto

### Get Toronto Venues from Foursquare

#### Define Credentials and Version

In [10]:
# IMPORTANT: You need to insert your own credentials here.

CLIENT_ID = ''
CLIENT_SECRET = ''
VERSION = '20200316'

#### Search for Venues near the Neighbourhoods

In [11]:
LIMIT = 100 # limit of number of venues returned
radius = 1500 # radius of venues search
categories = []
counter = []

# define GET Url
for lat, lng, nei in zip(pc_toronto['Latitude'], pc_toronto['Longitude'], pc_toronto['Neighbourhood']):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION, 
        lat,
        lng,
        radius, 
        LIMIT)

    get_result = requests.get(url).json()
    
    for item in get_result['response']['groups'][0]['items']:
        categories.append([nei, item['venue']['name'], item['venue']['categories'][0]['name']])
    
venues_df = pd.DataFrame(categories, columns=['Neighbourhood', 'Venue_Name', 'Venue_Category'])
print(venues_df.shape)
venues_df.head()

(6983, 3)


,Neighbourhood,Venue_Name,Venue_Category
0,"Rouge, Malvern",African Rainforest Pavilion,Zoo Exhibit
1,"Rouge, Malvern",Penguin Exhibit,Zoo Exhibit
2,"Rouge, Malvern",Orangutan Exhibit,Zoo Exhibit
3,"Rouge, Malvern",Gorilla Exhibit,Zoo Exhibit
4,"Rouge, Malvern",Canadiana exhibit,Zoo Exhibit


#### Descriptives

In [12]:
print('Number of neighbourhoods:', len(venues_df['Neighbourhood'].unique()))
print('Number of unique categories:', len(venues_df['Venue_Category'].unique()))
print('Number of unique venues:', len(venues_df['Venue_Name'].unique()))
print('Minimum number of venues in a neighbourhood:', venues_df.groupby('Neighbourhood').count().sort_values(by='Venue_Name', ascending=False).min()[0])
print('Maximum number of venues in a neighbourhood:', venues_df.groupby('Neighbourhood').count().sort_values(by='Venue_Name', ascending=False).max()[0])
print('Average number of venues in a neighbourhood:', round(venues_df.groupby('Neighbourhood').count().sort_values(by='Venue_Name', ascending=False).mean()[0]))

Number of neighbourhoods: 103
Number of unique categories: 343
Number of unique venues: 3188
Minimum number of venues in a neighbourhood: 3
Maximum number of venues in a neighbourhood: 100
Average number of venues in a neighbourhood: 68.0


### Analyze Neighbourhoods

#### Encoding the data

In [13]:
# one hot encoding
venues_encod = pd.get_dummies(venues_df['Venue_Category'])

# add neighborhood column back to dataframe
venues_encod['Neighbourhood'] = venues_df['Neighbourhood'] 

print(venues_encod.shape)
venues_encod.head()

(6983, 344)


,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shanghai Restaurant,

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [14]:
venues_group = venues_encod.groupby('Neighbourhood').mean().reset_index()

print(venues_group.shape)
venues_group.head()

(103, 344)


,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shangh

#### Print top three venue categories per neighbourhoods

In [15]:
for nei in venues_group['Neighbourhood']:
    temp = venues_group[venues_group['Neighbourhood'] == nei].transpose().reset_index()
    temp.columns = ['cat', 'freq']
    temp = temp.iloc[1:].sort_values('freq', ascending=False)
    temp = temp.astype({'freq': float}).round({'freq': 2})
    temp = temp.iloc[0:3].reset_index(drop=True)
    print('--------', nei, '--------')
    print(temp)
    print('\n')

-------- Adelaide, King, Richmond --------
           cat  freq
0  Coffee Shop  0.08
1        Hotel  0.06
2   Restaurant  0.05


-------- Agincourt --------
                    cat  freq
0    Chinese Restaurant  0.21
1                Bakery  0.04
2  Cantonese Restaurant  0.04


-------- Agincourt North, L'Amoreaux East, Milliken, Steeles East --------
                  cat  freq
0  Chinese Restaurant  0.17
1         Coffee Shop  0.07
2         Pizza Place  0.05


-------- Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown --------
                 cat  freq
0        Pizza Place  0.22
1      Grocery Store  0.17
2  Indian Restaurant  0.06


-------- Alderwood, Long Branch --------
              cat  freq
0  Discount Store  0.08
1        Pharmacy  0.08
2     Coffee Shop  0.08


-------- Bathurst Manor, Downsview North, Wilson Heights --------
              cat  freq
0     Coffee Shop  0.09
1            Park  0.06
2  Sandwich Place

-------- Ryerson, Garden District --------
           cat  freq
0  Coffee Shop  0.09
1   Restaurant  0.04
2    Gastropub  0.04


-------- Scarborough Village --------
                    cat  freq
0           Coffee Shop  0.12
1        Sandwich Place  0.09
2  Fast Food Restaurant  0.06


-------- Silver Hills, York Mills --------
           cat  freq
0  Coffee Shop  0.15
1         Park  0.07
2         Bank  0.07


-------- St. James Town --------
           cat  freq
0  Coffee Shop  0.08
1         Café  0.08
2   Restaurant  0.07


-------- Stn A PO Boxes 25 The Esplanade --------
                   cat  freq
0          Coffee Shop  0.06
1                 Café  0.05
2  Japanese Restaurant  0.04


-------- Studio District --------
                     cat  freq
0            Coffee Shop  0.06
1                Brewery  0.06
2  Vietnamese Restaurant  0.05


-------- The Annex, North Midtown, Yorkville --------
                  cat  freq
0         Coffee Shop  0.06
1  Italian Restaurant  0.

### Cluster Neighbourhoods

In [16]:
# set number of clusters
k = 5

venues_group_clust = venues_group.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(venues_group_clust)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 0, 0, 0, 0, 1, 1, 3, 1], dtype=int32)

#### Add clusters to the tables

In [17]:
pc_toronto = pc_toronto.sort_values(by='Neighbourhood')
pc_toronto['cluster'] = kmeans.labels_
venues_group['cluster'] = kmeans.labels_
venues_group_clust['cluster'] = kmeans.labels_

#### Create table grouped by cluster and average frequency of the cluster

In [18]:
cluster_group = venues_group_clust.groupby(by='cluster').mean().reset_index()

#### Print top 3 venue categories by cluster

In [19]:
for cl in cluster_group['cluster']:
    temp = cluster_group[cluster_group['cluster'] == cl].transpose().reset_index()
    temp.columns = ['cat', 'freq']
    temp = temp.iloc[1:].sort_values('freq', ascending=False)
    temp = temp.astype({'freq': float}).round({'freq': 2})
    temp = temp.iloc[0:3].reset_index(drop=True)
    print('-------- Cluster', cl, '--------')
    print(temp)
    print('\n')

-------- Cluster 0 --------
           cat  freq
0  Coffee Shop  0.10
1  Pizza Place  0.06
2         Park  0.04


-------- Cluster 1 --------
                  cat  freq
0         Coffee Shop  0.07
1                Park  0.05
2  Italian Restaurant  0.05


-------- Cluster 2 --------
                 cat  freq
0        Golf Course  0.67
1               Park  0.33
2  Accessories Store  0.00


-------- Cluster 3 --------
           cat  freq
0  Coffee Shop  0.08
1         Café  0.06
2   Restaurant  0.03


-------- Cluster 4 --------
                    cat  freq
0           Zoo Exhibit  0.50
1  Fast Food Restaurant  0.06
2                 Trail  0.06




### Visualize Clusters

In [20]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[lat_toronto, lng_toronto], zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat, lng, bor, nei, cluster in zip(pc_toronto['Latitude'], pc_toronto['Longitude'], pc_toronto['Borough'], pc_toronto['Neighbourhood'], pc_toronto['cluster']):
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup='{} ({}) cluster: {}'.format(nei, bor, cluster),
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.5).add_to(map_toronto)

map_toronto